In [5]:
%%sh
# run shell scripts to have an easy interface to experiment and get started with DPNE
# This script runs a series of tokenization, ngram extraction, and DP N-gram extraction using the parameters specified as arguments
# first argument = location of source file to extract DPNE from
SOURCE_DATASET="./dpne/normalized_rephrased_wildchat_data.json"

# second argument = output folder high level
OUTPUT_FOLDER="output_norm_wildchat10k_eps4"

# third argument = file extension - json or anything else.
FILE_EXTENSION="json"

# fourth argument = epsilon for DP
DP_EPSILON="4.0"

# fifth argument = the highest N in n-grams to DP-extract
NGRAM_SIZE_LIMIT="10"

#spark-submit dpne/tokenize_text.py -f json --ngrams $NGRAM_SIZE_LIMIT --max_num_tokens 400 --allow_multiple_ngrams 1 -i $SOURCE_DATASET -o ./$OUTPUT_FOLDER/tokenize_text -t $FILE_EXTENSION

#spark-submit dpne/split_ngrams.py --ngram_size $NGRAM_SIZE_LIMIT -i ./$OUTPUT_FOLDER/tokenize_text -o ./$OUTPUT_FOLDER/split_ngrams -f $FILE_EXTENSION -t $FILE_EXTENSION

# modified the hyperparams a bit
spark-submit dpne/extract_dpne.py --dp_epsilon $DP_EPSILON --dp_eta 0.1 --dp_delta 0.5 --contribution_limit 10 --persist_flags 11 --log_flags 00 --top_k 1 --delta_user_count 0 --ngram_size $NGRAM_SIZE_LIMIT --filter_one_side 0 --budget_distribute 1.0 --estimate_sample_size 0.8 -i ./$OUTPUT_FOLDER/split_ngrams -o ./$OUTPUT_FOLDER/dpne_sample -f $FILE_EXTENSION -t $FILE_EXTENSION

SystemLog:INFO:root:2024-09-20 17:01:11.301186	INFO	Read parameters...
SystemLog:INFO:root:2024-09-20 17:01:11.303190	INFO	Finished reading parameters.
SystemLog:INFO:root:2024-09-20 17:01:11.303319	INFO	2024-09-20 17:01:11


24/09/20 17:01:11 INFO SparkContext: Running Spark version 3.5.0
24/09/20 17:01:11 INFO SparkContext: OS info Linux, 5.15.153.1-microsoft-standard-WSL2, amd64
24/09/20 17:01:11 INFO SparkContext: Java version 17.0.8.1
24/09/20 17:01:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/20 17:01:11 INFO ResourceUtils: ==============================================================
24/09/20 17:01:11 INFO ResourceUtils: No custom resources configured for spark.driver.
24/09/20 17:01:11 INFO ResourceUtils: ==============================================================
24/09/20 17:01:11 INFO SparkContext: Submitted application: DPNE_/home/jovyan/differentially-private-ngram-extraction/dpne/extract_dpne.py_2024-09-20 17:01:11
24/09/20 17:01:11 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, scr

SystemLog:INFO:root:2024-09-20 17:01:13.054075	INFO	Input file type is json
SystemLog:INFO:root:2024-09-20 17:01:18.555264	INFO	Sampling top k n-grams instead of random sampling from each user..
SystemLog:INFO:root:2024-09-20 17:01:19.583556	INFO	total 1-grams: 138098
SystemLog:INFO:root:2024-09-20 17:01:21.037407	INFO	dstinct # of n-grams: 12179
SystemLog:INFO:root:2024-09-20 17:01:21.236128	INFO	selected sigma value for Gaussian: 0.397390428948983
SystemLog:INFO:root:2024-09-20 17:01:21.238963	INFO	Params Delta_0=10, delta=5.00e-01, g_param=1.2566588758300987, g_rho=2.7107759957247493


24/09/20 17:01:26 WARN CacheManager: Asked to cache already cached data.


SystemLog:INFO:root:2024-09-20 17:01:26.945301	INFO	Retrieved 1-gram: 520
SystemLog:INFO:root:2024-09-20 17:01:26.945538	INFO	Trying to write to ./output_norm_wildchat10k_eps4/dpne_sample/1gram
SystemLog:INFO:root:2024-09-20 17:01:27.368453	INFO	Input file type is json
SystemLog:INFO:root:2024-09-20 17:01:27.535463	INFO	Input file type is json
SystemLog:INFO:root:2024-09-20 17:01:27.641649	INFO	Input file type is json
SystemLog:INFO:root:2024-09-20 17:01:27.849701	INFO	Sampling top k n-grams instead of random sampling from each user..
SystemLog:INFO:root:2024-09-20 17:01:28.069944	INFO	total 2-grams: 128201
SystemLog:INFO:root:2024-09-20 17:01:28.517275	INFO	n-grams to check: 216225


...

24/09/20 17:02:22 WARN CacheManager: Asked to cache already cached data.


SystemLog:INFO:root:2024-09-20 17:02:22.968289	INFO	Retrieved 9-gram: 3
SystemLog:INFO:root:2024-09-20 17:02:22.968668	INFO	Trying to write to ./output_norm_wildchat10k_eps4/dpne_sample/9gram
SystemLog:INFO:root:2024-09-20 17:02:23.138981	INFO	Input file type is json
SystemLog:INFO:root:2024-09-20 17:02:23.243706	INFO	Input file type is json
SystemLog:INFO:root:2024-09-20 17:02:23.348922	INFO	Input file type is json
SystemLog:INFO:root:2024-09-20 17:02:23.474257	INFO	Sampling top k n-grams instead of random sampling from each user..
SystemLog:INFO:root:2024-09-20 17:02:23.599903	INFO	total 10-grams: 50593
SystemLog:INFO:root:2024-09-20 17:02:23.828742	INFO	n-grams to check: 930


24/09/20 17:02:23 WARN ExtractPythonUDFFromJoinCondition: The join condition:isnotnull(extract_ngrams_to_validate(concat(ngrams#6863,  , ngrams#6885))#6931) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.


SystemLog:INFO:root:2024-09-20 17:02:24.354754	INFO	estimated valid # of n-grams: 1
SystemLog:INFO:root:2024-09-20 17:02:24.356300	INFO	selected sigma value for Gaussian: 0.397390428948983
SystemLog:INFO:root:2024-09-20 17:02:24.357169	INFO	Params Delta_0=10, delta=5.00e-01, g_param=1.2566588758300987, g_rho=1.6104731496755806
SystemLog:INFO:root:2024-09-20 17:02:24.672985	INFO	dstinct # of n-grams: 50270
SystemLog:INFO:root:2024-09-20 17:02:25.313382	INFO	actual valid # of n-grams: 0
SystemLog:INFO:root:2024-09-20 17:02:26.273882	INFO	adding spurious n-grams: 0
SystemLog: Traceback (most recent call last):
SystemLog:   File "/home/jovyan/differentially-private-ngram-extraction/dpne/extract_dpne.py", line 383, in execute
SystemLog:     ngrams_extracted.persist()
SystemLog:     ^^^^^^^^^^^^^^^^^^^^^^^^
SystemLog: AttributeError: SystemLog:**Exception message scrubbed**
Traceback (most recent call last):
  File "/home/jovyan/differentially-private-ngram-extraction/dpne/extract_dpne.py", 

CalledProcessError: Command 'b'# run shell scripts to have an easy interface to experiment and get started with DPNE\n# This script runs a series of tokenization, ngram extraction, and DP N-gram extraction using the parameters specified as arguments\n# first argument = location of source file to extract DPNE from\nSOURCE_DATASET="./dpne/normalized_rephrased_wildchat_data.json"\n\n# second argument = output folder high level\nOUTPUT_FOLDER="output_norm_wildchat10k_eps4"\n\n# third argument = file extension - json or anything else.\nFILE_EXTENSION="json"\n\n# fourth argument = epsilon for DP\nDP_EPSILON="4.0"\n\n# fifth argument = the highest N in n-grams to DP-extract\nNGRAM_SIZE_LIMIT="10"\n\n#spark-submit dpne/tokenize_text.py -f json --ngrams $NGRAM_SIZE_LIMIT --max_num_tokens 400 --allow_multiple_ngrams 1 -i $SOURCE_DATASET -o ./$OUTPUT_FOLDER/tokenize_text -t $FILE_EXTENSION\n\n#spark-submit dpne/split_ngrams.py --ngram_size $NGRAM_SIZE_LIMIT -i ./$OUTPUT_FOLDER/tokenize_text -o ./